In [91]:
from rethon.ensemble_generation import EnsembleGenerator, SimpleEnsembleGenerator
from rethon.ensemble_generation import create_random_argument_list, standard_model_params_varied_alphas
from rethon.ensemble_generation import random_position_as_set, create_random_arguments2
from rethon.model import StandardPosition, DAGDialecticalStructure, StandardGlobalReflectiveEquilibrium
from os import getcwd
import concurrent.futures
import time

In [104]:
def experiment(options):
    arguments_list = options[0]
    coms_list = options [1]
    implementations = options [2]
    
    start = time.perf_counter()

    model_params = [{
        'neighbourhood_depth': 1,
        'faithfulness_penalties': [0.0, 0.0, 1.0, 1.0],
        'account_penalties': [0.0, 0.3, 1.0, 1.0],
        'weights': {'account': 0.35, 'faithfulness': 0.1, 'systematicity': 0.55}
    }]

    ensemble_gen = SimpleEnsembleGenerator(arguments_list = arguments_list, 
                                     n_sentence_pool = n,
                                     initial_commitments_list = coms_list,
                                     model_parameters_list = model_params,
                                     implementations = implementations)


    # removing items we don't need
    ensemble_gen.remove_item('account_penalties')
    ensemble_gen.remove_item('faithfulness_penalties')
    ensemble_gen.remove_item('weight_account')
    ensemble_gen.remove_item('weight_systematicity')
    ensemble_gen.remove_item('weight_faithfulness')
    ensemble_gen.remove_item('n_random_choices')
    ensemble_gen.remove_item('n_init_coms_min_ax_base')

    # defining the function for the global Optima
    def data_global_optima(ensemble_generator):
        re_model = ensemble_generator.reflective_equilibrium()
        if re_model.model_name() == "StandardModel":
            return re_model.global_optima(ensemble_generator.initial_commitments())

    ensemble_gen.add_item('global optima', data_global_optima)
    argument_size = len(arguments_list[0])
    #max_premises = 

    ensemble_gen.ensemble_items_to_csv(
                                  output_file_name = f'data_output_args-{argument_size}_model-{implementations[0]["reflective_equilibrium_class_name"][8:13]}.csv',
                                  output_dir_name = getcwd(),
                                  archive = False, # save the csv as archived tar.gz
                                  save_preliminary_results = True, # will create preliminary csv-data sets 
                                  preliminary_results_interval = 50)
    
    return time.perf_counter()-start

In [105]:
def createStruct(argument_size, max_premises):
    # Sentence pool
    n= 7
    # Two random dialectical structures with 7 arguments each
    n_dialectical_structures = 5
    arguments_list = [create_random_arguments2(n_sentences=n, 
                                               n_arguments=argument_size,
                                               n_max_premises=max_premises) for i in range(n_dialectical_structures)]
    # Two random initial commitments
    n_positions = 2
    coms_list = [random_position_as_set(n_sentences = n) for i in range(n_positions)]
    local_impl = [{'module_name': 'rethon.model',
               'position_class_name':'StandardPosition',
               'dialectical_structure_class_name': 'DAGDialecticalStructure',
               'reflective_equilibrium_class_name': 'StandardLocalReflectiveEquilibrium'
               }]
    global_impl = [{'module_name': 'rethon.model',
                'position_class_name':'StandardPosition',
                'dialectical_structure_class_name': 'DAGDialecticalStructure',
                'reflective_equilibrium_class_name': 'StandardGlobalReflectiveEquilibrium'}]
    return [[arguments_list, coms_list, local_impl],[arguments_list, coms_list, global_impl]]



with concurrent.futures.ProcessPoolExecutor() as executor:
    times = executor.map(experiment, createStruct(2,1))
    for time in times:
         print(time)

[[6, 2], [5, -6]][[6, 2], [5, -6]]



NameError: name 'max_premises' is not defined